In [1]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, roc_curve, auc

# 加载训练集和测试集
train_data = pd.read_csv("/grp01/saas_lqqu/runxi/lightgbm/train_processed_features.csv")
test_data = pd.read_csv("/grp01/saas_lqqu/runxi/lightgbm/test_processed_features.csv")

# 确认目标列名称（假设分类任务目标列为'target'）
#TARGET_COL = 'Claim_Amount'

In [2]:
train_data = train_data.loc[:, ~train_data.columns.str.contains('\?')]
test_data = test_data.loc[:, ~test_data.columns.str.contains('\?')]

<>:1: SyntaxWarning: invalid escape sequence '\?'
<>:2: SyntaxWarning: invalid escape sequence '\?'
<>:1: SyntaxWarning: invalid escape sequence '\?'
<>:2: SyntaxWarning: invalid escape sequence '\?'
/tmp/ipykernel_2949221/1692780308.py:1: SyntaxWarning: invalid escape sequence '\?'
  train_data = train_data.loc[:, ~train_data.columns.str.contains('\?')]
/tmp/ipykernel_2949221/1692780308.py:2: SyntaxWarning: invalid escape sequence '\?'
  test_data = test_data.loc[:, ~test_data.columns.str.contains('\?')]


In [3]:
train_label = pd.read_csv("/grp01/saas_lqqu/runxi/lightgbm/train_labels.csv")
test_label = pd.read_csv("/grp01/saas_lqqu/runxi/lightgbm/test_labels.csv")

In [7]:
import lightgbm as lgb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split


# 将数据转换为 LightGBM 数据集格式
train_dataset = lgb.Dataset(train_data, label=train_label)
test_dataset = lgb.Dataset(test_data, label=test_label, reference=train_dataset)


In [8]:
params = {
    "objective": "binary",
    "metric": "auc",
    "boosting_type": "gbdt",
    "num_leaves": 31,
    "learning_rate": 0.05,
    "feature_fraction": 0.9,
    "verbose": -1
    
}

'''params = {
    "objective": "binary",
    "metric": "auc",
    "learning_rate": 0.1,
    "num_leaves": 255,
    "num_trees": 500,
    "min_data_in_leaf": 100,
    "min_sum_hessian_in_leaf": 0.0,
    "num_threads": 16,
    "max_bin": 255,   # 增加直方图分桶数
    #"device_type": "gpu"  # 使用GPU加速
}'''

# 初始化时间和AUC记录
start_time = time.time()
time_points_EFB = []
auc_values_EFB = []

# 自定义回调函数记录时间和AUC
def record_time_auc(env):
    current_time = time.time() - start_time
    time_points_EFB.append(current_time)
    # 遍历评估结果，获取验证集AUC
    for eval_result in env.evaluation_result_list:
        if eval_result[0] == "valid" and eval_result[1] == "auc":
            auc_values_EFB.append(eval_result[2])

# 训练模型
gbm = lgb.train(
    params,
    train_dataset,
    num_boost_round=100,
    valid_sets=[test_dataset],
    valid_names=["valid"],
    callbacks=[record_time_auc, lgb.log_evaluation(period=1)]
)

# 绘制时间-AUC曲线
'''plt.figure(figsize=(10, 6))
plt.plot(time_points, auc_values, "b-", linewidth=2)
plt.xlabel("Training Time (seconds)", fontsize=12)
plt.ylabel("Validation AUC", fontsize=12)
plt.title("Time-AUC Curve during Training", fontsize=14)
plt.grid(True, linestyle="--", alpha=0.7)
plt.tight_layout()
plt.show()'''

[1]	valid's auc: 0.545823
[2]	valid's auc: 0.546426
[3]	valid's auc: 0.54657
[4]	valid's auc: 0.547176
[5]	valid's auc: 0.547675
[6]	valid's auc: 0.547829
[7]	valid's auc: 0.547748
[8]	valid's auc: 0.54787
[9]	valid's auc: 0.547988
[10]	valid's auc: 0.547987
[11]	valid's auc: 0.548024
[12]	valid's auc: 0.54823
[13]	valid's auc: 0.548277
[14]	valid's auc: 0.548454
[15]	valid's auc: 0.548541
[16]	valid's auc: 0.548521
[17]	valid's auc: 0.548882
[18]	valid's auc: 0.548892
[19]	valid's auc: 0.549007
[20]	valid's auc: 0.549054
[21]	valid's auc: 0.549234
[22]	valid's auc: 0.54915
[23]	valid's auc: 0.54918
[24]	valid's auc: 0.549369
[25]	valid's auc: 0.549358
[26]	valid's auc: 0.549404
[27]	valid's auc: 0.549404
[28]	valid's auc: 0.549452
[29]	valid's auc: 0.549414
[30]	valid's auc: 0.549347
[31]	valid's auc: 0.54939
[32]	valid's auc: 0.549394
[33]	valid's auc: 0.549369
[34]	valid's auc: 0.549413
[35]	valid's auc: 0.549441
[36]	valid's auc: 0.549542
[37]	valid's auc: 0.549587
[38]	valid's auc

'plt.figure(figsize=(10, 6))\nplt.plot(time_points, auc_values, "b-", linewidth=2)\nplt.xlabel("Training Time (seconds)", fontsize=12)\nplt.ylabel("Validation AUC", fontsize=12)\nplt.title("Time-AUC Curve during Training", fontsize=14)\nplt.grid(True, linestyle="--", alpha=0.7)\nplt.tight_layout()\nplt.show()'

In [9]:
params = {
    "objective": "binary",
    "metric": "auc",
    "boosting_type": "goss",
    "num_leaves": 31,
    "learning_rate": 0.05,
    "feature_fraction": 0.9,
    "verbose": -1
    
}

'''params = {
    "objective": "binary",
    "metric": "auc",
    "learning_rate": 0.1,
    "num_leaves": 255,
    "num_trees": 500,
    "min_data_in_leaf": 100,
    "min_sum_hessian_in_leaf": 0.0,
    "num_threads": 16,
    "max_bin": 255,   # 增加直方图分桶数
    #"device_type": "gpu"  # 使用GPU加速
}'''

# 初始化时间和AUC记录
start_time = time.time()
time_points_Full = []
auc_values_Full = []

# 自定义回调函数记录时间和AUC
def record_time_auc(env):
    current_time = time.time() - start_time
    time_points_Full.append(current_time)
    # 遍历评估结果，获取验证集AUC
    for eval_result in env.evaluation_result_list:
        if eval_result[0] == "valid" and eval_result[1] == "auc":
            auc_values_Full.append(eval_result[2])

# 训练模型
gbm = lgb.train(
    params,
    train_dataset,
    num_boost_round=100,
    valid_sets=[test_dataset],
    valid_names=["valid"],
    callbacks=[record_time_auc, lgb.log_evaluation(period=1)]
)

# 绘制时间-AUC曲线
'''plt.figure(figsize=(10, 6))
plt.plot(time_points, auc_values, "b-", linewidth=2)
plt.xlabel("Training Time (seconds)", fontsize=12)
plt.ylabel("Validation AUC", fontsize=12)
plt.title("Time-AUC Curve during Training", fontsize=14)
plt.grid(True, linestyle="--", alpha=0.7)
plt.tight_layout()
plt.show()'''

[1]	valid's auc: 0.545823
[2]	valid's auc: 0.546426
[3]	valid's auc: 0.54657
[4]	valid's auc: 0.547176
[5]	valid's auc: 0.547675
[6]	valid's auc: 0.547829
[7]	valid's auc: 0.547748
[8]	valid's auc: 0.54787
[9]	valid's auc: 0.547988
[10]	valid's auc: 0.547987
[11]	valid's auc: 0.548024
[12]	valid's auc: 0.54823
[13]	valid's auc: 0.548277
[14]	valid's auc: 0.548454
[15]	valid's auc: 0.548541
[16]	valid's auc: 0.548521
[17]	valid's auc: 0.548882
[18]	valid's auc: 0.548892
[19]	valid's auc: 0.549007
[20]	valid's auc: 0.549054
[21]	valid's auc: 0.549243
[22]	valid's auc: 0.54934
[23]	valid's auc: 0.549535
[24]	valid's auc: 0.549492
[25]	valid's auc: 0.549464
[26]	valid's auc: 0.54953
[27]	valid's auc: 0.549496
[28]	valid's auc: 0.549546
[29]	valid's auc: 0.549424
[30]	valid's auc: 0.549454
[31]	valid's auc: 0.549403
[32]	valid's auc: 0.549527
[33]	valid's auc: 0.54952
[34]	valid's auc: 0.549611
[35]	valid's auc: 0.549614
[36]	valid's auc: 0.549637
[37]	valid's auc: 0.549567
[38]	valid's auc

'plt.figure(figsize=(10, 6))\nplt.plot(time_points, auc_values, "b-", linewidth=2)\nplt.xlabel("Training Time (seconds)", fontsize=12)\nplt.ylabel("Validation AUC", fontsize=12)\nplt.title("Time-AUC Curve during Training", fontsize=14)\nplt.grid(True, linestyle="--", alpha=0.7)\nplt.tight_layout()\nplt.show()'

In [10]:
params = {
    "objective": "binary",
    "metric": "auc",
    "boosting_type": "gbdt",
    "num_leaves": 31,
    "learning_rate": 0.05,
    "feature_fraction": 0.9,
    "verbose": -1,
    'enable_bundle': False,
    
}

train_dataset_ = lgb.Dataset(train_data, label=train_label)
test_dataset_ = lgb.Dataset(test_data, label=test_label, reference=train_dataset)

'''params = {
    "objective": "binary",
    "metric": "auc",
    "learning_rate": 0.1,
    "num_leaves": 255,
    "num_trees": 500,
    "min_data_in_leaf": 100,
    "min_sum_hessian_in_leaf": 0.0,
    "num_threads": 16,
    "max_bin": 255,   # 增加直方图分桶数
    #"device_type": "gpu"  # 使用GPU加速
}'''

# 初始化时间和AUC记录
start_time = time.time()
time_points_base = []
auc_values_base = []

# 自定义回调函数记录时间和AUC
def record_time_auc(env):
    current_time = time.time() - start_time
    time_points_base.append(current_time)
    # 遍历评估结果，获取验证集AUC
    for eval_result in env.evaluation_result_list:
        if eval_result[0] == "valid" and eval_result[1] == "auc":
            auc_values_base.append(eval_result[2])

# 训练模型
gbm = lgb.train(
    params,
    train_dataset_,
    num_boost_round=100,
    valid_sets=[test_dataset_],
    valid_names=["valid"],
    callbacks=[record_time_auc, lgb.log_evaluation(period=1)]
)

# 绘制时间-AUC曲线
'''plt.figure(figsize=(10, 6))
plt.plot(time_points, auc_values, "b-", linewidth=2)
plt.xlabel("Training Time (seconds)", fontsize=12)
plt.ylabel("Validation AUC", fontsize=12)
plt.title("Time-AUC Curve during Training", fontsize=14)
plt.grid(True, linestyle="--", alpha=0.7)
plt.tight_layout()
plt.show()'''

[1]	valid's auc: 0.545823
[2]	valid's auc: 0.546426
[3]	valid's auc: 0.54657
[4]	valid's auc: 0.547176
[5]	valid's auc: 0.547675
[6]	valid's auc: 0.547829
[7]	valid's auc: 0.547748
[8]	valid's auc: 0.54787
[9]	valid's auc: 0.547988
[10]	valid's auc: 0.547987
[11]	valid's auc: 0.548024
[12]	valid's auc: 0.54823
[13]	valid's auc: 0.548277
[14]	valid's auc: 0.548454
[15]	valid's auc: 0.548541
[16]	valid's auc: 0.548521
[17]	valid's auc: 0.548882
[18]	valid's auc: 0.548892
[19]	valid's auc: 0.549007
[20]	valid's auc: 0.549054
[21]	valid's auc: 0.549234
[22]	valid's auc: 0.54915
[23]	valid's auc: 0.54918
[24]	valid's auc: 0.549369
[25]	valid's auc: 0.549358
[26]	valid's auc: 0.549404
[27]	valid's auc: 0.549404
[28]	valid's auc: 0.549452
[29]	valid's auc: 0.549414
[30]	valid's auc: 0.549347
[31]	valid's auc: 0.54939
[32]	valid's auc: 0.549394
[33]	valid's auc: 0.549369
[34]	valid's auc: 0.549413
[35]	valid's auc: 0.549441
[36]	valid's auc: 0.549542
[37]	valid's auc: 0.549587
[38]	valid's auc

'plt.figure(figsize=(10, 6))\nplt.plot(time_points, auc_values, "b-", linewidth=2)\nplt.xlabel("Training Time (seconds)", fontsize=12)\nplt.ylabel("Validation AUC", fontsize=12)\nplt.title("Time-AUC Curve during Training", fontsize=14)\nplt.grid(True, linestyle="--", alpha=0.7)\nplt.tight_layout()\nplt.show()'

In [18]:
print(len(time_points_base))
print(len(time_points_Full))
print(len(time_points_EFB))

100
100
100


In [11]:
import pickle

with open('time_points_base.pkl', 'wb') as file:
    pickle.dump(time_points_base, file)

with open('time_points_Full.pkl', 'wb') as file:
    pickle.dump(time_points_Full, file)

with open('time_points_EFB.pkl', 'wb') as file:
    pickle.dump(time_points_EFB, file)

with open('auc_values_base.pkl', 'wb') as file:
    pickle.dump(auc_values_base, file)

with open('auc_values_Full.pkl', 'wb') as file:
    pickle.dump(auc_values_Full, file)

with open('auc_values_EFB.pkl', 'wb') as file:
    pickle.dump(auc_values_EFB, file)